In [ ]:
!pip install transformers


In [2]:
import transformers


In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("AswiN037/tamil-Roberta-small")

model = AutoModelForMaskedLM.from_pretrained("AswiN037/tamil-Roberta-small")

Downloading: 100%|██████████| 1.10k/1.10k [00:00<00:00, 1.12MB/s]
Downloading: 100%|██████████| 590k/590k [00:00<00:00, 666kB/s] 
Downloading: 100%|██████████| 331k/331k [00:00<00:00, 512kB/s] 
Downloading: 100%|██████████| 636/636 [00:00<00:00, 616kB/s]
Downloading: 100%|██████████| 256M/256M [00:25<00:00, 10.7MB/s]   


In [4]:
from transformers import pipeline

fill = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [7]:
text = ["சிரிப்பு இல்லாத வாழ்க்கை சிறகு இல்லாத பறவைக்கு சமம்."]
masked_text = [f"சிரிப்பு இல்லாத {fill.tokenizer.mask_token} சிறகு இல்லாத பறவைக்கு சமம்."]
fill(masked_text)


[{'score': 0.16797690093517303,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'சிரிப்பு இல்லாத சிறகு இல்லாத பறவைக்கு சமம்.'},
 {'score': 0.0002627014182507992,
  'token': 6336,
  'token_str': ' link',
  'sequence': 'சிரிப்பு இல்லாத link சிறகு இல்லாத பறவைக்கு சமம்.'},
 {'score': 0.00021842599380761385,
  'token': 20152,
  'token_str': 'Search',
  'sequence': 'சிரிப்பு இல்லாதSearch சிறகு இல்லாத பறவைக்கு சமம்.'},
 {'score': 0.00021697136980947107,
  'token': 10646,
  'token_str': ' youth',
  'sequence': 'சிரிப்பு இல்லாத youth சிறகு இல்லாத பறவைக்கு சமம்.'},
 {'score': 0.00020036357454955578,
  'token': 6192,
  'token_str': 'ரகவ',
  'sequence': 'சிரிப்பு இல்லாதரகவ சிறகு இல்லாத பறவைக்கு சமம்.'}]

In [8]:
text = "நான் கணினிப் பொறியியல் படிக்கிறேன்"
masked_text = f"நான் கணினிப் பொறியியல் {fill.tokenizer.mask_token}"
fill(masked_text)


[{'score': 0.1638135015964508,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'நான் கணினிப் பொறியியல்'},
 {'score': 0.0002775640459731221,
  'token': 462,
  'token_str': ' அதன',
  'sequence': 'நான் கணினிப் பொறியியல் அதன'},
 {'score': 0.00020209793001413345,
  'token': 2926,
  'token_str': 'ளனம',
  'sequence': 'நான் கணினிப் பொறியியல்ளனம'},
 {'score': 0.0002020162355620414,
  'token': 7018,
  'token_str': 'public',
  'sequence': 'நான் கணினிப் பொறியியல்public'},
 {'score': 0.00019224980496801436,
  'token': 6336,
  'token_str': ' link',
  'sequence': 'நான் கணினிப் பொறியியல் link'}]

In [10]:
fp = r'../dataset/tamilqa.json'

In [9]:
import pandas as pd
from tqdm.auto import tqdm
import numpy

In [11]:
with open(fp, "r") as read_file:
  qa = pd.read_json(read_file)

In [13]:
# qa = qa.reset_index()  # make sure indexes pair with number of rows

# dataset = [{'context' : row['context'], 
#             'id' : row['qas'][0]['id'], 
#             'question':  row['qas'][0]['question'], 
#             'answers':  row['qas'][0]['answers']} for i, row in tqdm(qa.iterrows())]
# dataset[0]

In [16]:
qa.iloc[0]

index                                                           0
context         ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...
question                     மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
answer_text                                                   206
answer_start                                                   53
Name: 0, dtype: object

In [21]:
from datasets import Dataset

In [24]:
dataset = Dataset.from_pandas(qa)

In [26]:
dataset

Dataset({
    features: ['index', 'context', 'question', 'answer_text', 'answer_start'],
    num_rows: 367
})

In [ ]:
query = dataset[0]['question']
query

docs = [ ]
count = 0
for item in dataset:
    if count < 10:
        docs.append(item['context'])
    else:
        break

docs

In [42]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.logits #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings

#Encode query and docs
query_emb = encode(query)
doc_emb = encode(docs)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_similar_contexts(context_embeddings, question_embedding):
  similarity_score = []
  for em in tqdm(context_embeddings):
    cos = torch.nn.CosineSimilarity(dim=0)
    output = cos(question_embedding, em)
    similarity_score.append((len(similarity_score), output))
  return similarity_score

In [ ]:
sim_scores = retrieve_similar_contexts(doc_emb, query_emb[0])
sim_scores